## 시험 인증: 데이터 품질 성능(정규성 위배율) 및 데이터 품질 향상도

정규성 위배율 측정 및 보정 도구 라이브러리 Import 

In [ ]:
import norm_violation as nv

### 시험 항목: 데이터 품질 성능(정규성 위배율)

height_org.csv (공공데이터포털 제공 전국고등학생 키(신장) 데이터) 의 정규성 위배율 측정

In [ ]:
nv.get_normality_violation('height_org.csv')

### 시험 항목: 데이터 품질 향상도(정규성 위배율)

보정 전 데이터셋(height-raw.csv)의 정규성 위배율 측정

In [ ]:
nv.get_normality_violation('height-raw.csv')

정규성 위배율 보정 모듈 수행. 입력인자(보정전 데이터셋 파일명, 보정후 결과 데이터셋 파일명)

In [ ]:
nv.correct_normality_violation('height-raw.csv', 'height-good.csv')

보정된 데이터셋의 정규성 위배율 측정

In [ ]:
nv.get_normality_violation('height-good.csv')